In [ ]:
import matplotlib.pyplot as plt
import matplotlib
from sklearn import preprocessing
import numpy as np
import json
from tabulate import tabulate
import random

In [ ]:
LETTERS = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

CLASSES = []
for i in range(0, len(LETTERS)):
    CLASSES.append(i)
    
#print(CLASSES)

FILES = ["User1_iznad", "User2_iznad"]
#FILES = ["User1_pored", "User2_pored"]

In [ ]:
data = []
d = []
for_print = []

for fn in FILES:
    with open("Dataset/" + fn + '.json', 'r') as file:
        f = json.load(file)

        for letter in f:
            l = letter["letter"]
            c = LETTERS.index(l) + 1
            t = letter["t"]
            num = len(letter["fingerprint"])
            points = letter["fingerprint"]

            d = [c, l, t, num, points]
            data.append(d)
            d = []

            ar = [c, l, t, num]
            for_print.append(ar)
            
print(tabulate(for_print, headers=["Class", "Letter", "Time", "Number of points"]))
print(len(data))

In [ ]:
all_points = 0

for d in data:
    all_points += d[3]

average_points = all_points // len(data)
print("Average number of points: " + str(average_points))
print("--------------------------------")

for d in data:
    letter_points = d[3]
    difference = letter_points - average_points
    
    if difference > 0:
        while difference > 0:
            random_index = random.randint(0, d[3]-1)
            
            d[4].pop(random_index)
            d[3] -= 1
            
            difference -= 1
        
    elif difference < 0:
        while difference < 0:
            before_index = random.randint(0, d[3]-2)
            after_index = before_index + 1
            
            new_x = (d[4][before_index][0] + d[4][after_index][0]) / 2
            new_y = (d[4][before_index][1] + d[4][after_index][1]) / 2
            new_z = (d[4][before_index][2] + d[4][after_index][2]) / 2
            
            new_point = [new_x, new_y, new_z]
            
            d[4].insert(after_index, new_point)
            d[3] += 1
                
            difference += 1

In [ ]:
dataset = []
classes = []
labels = []

for d in data:
    classes.append(d[0])
    labels.append(d[1])
    dataset.append(d[4])

In [ ]:
norm_dataset = []

for d in dataset:
    scaler = preprocessing.MinMaxScaler()
    
    norm_d = scaler.fit_transform(d)
    norm_dataset.append(norm_d)
    norm_d = []

print(norm_dataset[0])

In [ ]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
import pandas as pd

norm_dataset = np.array(norm_dataset)
norm_dataset = norm_dataset.reshape(norm_dataset.shape[0], -1)

tsne = TSNE(n_components=3).fit_transform(norm_dataset)
print(tsne)

#pca_model = PCA(n_components=2)
#pca_model.fit(norm_dataset)
#pca = pca_model.transform(norm_dataset)

#ica_model = FastICA(n_components=3, random_state=0, whiten='unit-variance')
#ica= ica_model.fit_transform(norm_dataset)
#print(ica.shape)

In [ ]:
data = {'x':tsne[:,0], 'y':tsne[:,1], 'dig':classes, 'label':labels}
df = pd.DataFrame(data)
df.head()

In [ ]:
import colorcet as cc
import seaborn as sns

n = len(LETTERS)
print(n)

palette = sns.color_palette(cc.glasbey, n_colors=n)
image_name = input()

In [ ]:
plt.rcParams['figure.figsize'] = [8, 8]

sns.scatterplot(data=df, x="x", y="y", hue="label", style="label", palette=palette, s=50)

plt.legend(bbox_to_anchor=(1.15, 1), borderaxespad=0)
plt.savefig("Slike/" + image_name + ".png", bbox_inches='tight')
plt.show()